Techniques Demonstrated:
  1) Saving and Loading pre-trained Model Weights
  2) Saving and Loading NumPy objects (of images) - much faster!
  3) Early Stopping of CNN Training
  4) Saving and Loading Model Training history, and Visualizing (preserved) Model Training History
To-learn:
  1) Saving model probabilities in a nice way (for ensembling)

In [1]:
# Loading Common Packages for Our Team
import pickle as pickle
import numpy as np
import os
import pandas as pd
import random
from tqdm import tqdm
# import xgboost as xgb   # Fan Fei does not have XGBoost on Windows
import pickle as pickle
import cv2
import datetime as dt
print("------Imports complete-----")

------Imports complete-----


In [2]:
# Import Model Specific packages
from keras.preprocessing.image import img_to_array, load_img

import keras as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

# import packages for InceptionV3
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model

from multiprocessing import Pool, cpu_count

# callback for saving models, early stopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

# for plotting model training history
import matplotlib.pyplot as plt

# os.chdir('C:/deep_learning') # This is where the input dataset is stored
# os.getcwd()
print("------Fan Fei's Imports Complete-----")

Using TensorFlow backend.


------Fan Fei's Imports Complete-----


In [3]:
random_seed = 987654321
random.seed(random_seed)
np.random.seed(random_seed)

# train_path = "c:/deep_learning/kaggle/amazon/train/"
# test_path = "c:/deep_learning/kaggle/amazon/test/"
train_path = "f:/kaggle/amazon/train-jpg/"
test_path = "f:/kaggle/amazon/test-jpg/"
# train_path = "/mnt/f/kaggle/amazon/train-jpg/"
# test_path = "/mnt/f/kaggle/amazon/test-jpg/"

# train = pd.read_csv("c:/deep_learning/kaggle/amazon/train_v2.csv")
train = pd.read_csv("c:/deep_learning/kaggle/amazon/train_v2.csv")[:1280]      # take only the first 100 images
# train = pd.read_csv("c:/deep_learning/kaggle/amazon/train_v2_short.csv")
# test =  pd.read_csv("c:/deep_learning/kaggle/amazon/sample_submission_v2.csv")
test =  pd.read_csv("c:/deep_learning/kaggle/amazon/sample_submission_v2.csv")[:1280]
# test =  pd.read_csv("c:/deep_learning/kaggle/amazon/sample_submission_v2_short.csv")

#Location for saving serialized objects
obj_save_path = "c:/deep_learning/kaggle/amazon/model/"
subm_output_path = "c:/deep_learning/kaggle/amazon/submissions/"
print("------Paths Built-----")

------Paths Built-----


In [4]:
def keras_image_prep(df, data_path, name, dim, labels):
    x_out = []
    y_out = []

    # flatten = lambda l: [item for sublist in l for item in sublist]
    # labels = list(set(flatten([l.split(' ') for l in df['tags'].values])))

    label_map = {l: i for i, l in enumerate(labels)}
    # inv_label_map = {i: l for l, i in label_map.items()}

    for image_name, tags in tqdm(df.values, miniters=1000):
        img = img_to_array(load_img(data_path + image_name + '.jpg', target_size=[dim,dim]))
        targets = np.zeros(17)
        for t in tags.split(' '):
            targets[label_map[t]] = 1 
        x_out.append(img)
        y_out.append(targets)
    
    x_out = np.array(x_out, np.float16) / 255.
    y_out = np.array(y_out, np.uint8)

    print(x_out.shape)
    print(y_out.shape)
    return x_out, y_out

In [5]:
input_dim = 299
labels = train['tags'].str.get_dummies(sep=' ').columns

In [6]:
x_train0, y_train0 = keras_image_prep(train, train_path, 'train', input_dim, labels)
# this is to provide a local validation F-score
# split = 35000
split = 1024 # = 1280*0.80
x_train, x_valid, y_train, y_valid = x_train0[:split], x_train0[split:], y_train0[:split], y_train0[split:]

# Save out the image NumPy to disk
np.save('f:/kaggle/amazon/x_train_incv3.npy', x_train)
np.save('f:/kaggle/amazon/y_train_incv3.npy', y_train)
np.save('f:/kaggle/amazon/x_valid_incv3.npy', x_valid)
np.save('f:/kaggle/amazon/y_valid_incv3.npy', y_valid)

# can use the following block after the image has been saved as NumPy
# x_train = np.load('f:/kaggle/amazon/x_train_incv3.npy')
# y_train = np.load('f:/kaggle/amazon/y_train_incv3.npy')
# x_valid = np.load('f:/kaggle/amazon/x_valid_incv3.npy')
# y_valid = np.load('f:/kaggle/amazon/y_valid_incv3.npy')

100%|██████████████████████████████████████████████████████████████████████████████| 1280/1280 [00:52<00:00, 24.61it/s]


(1280, 299, 299, 3)
(1280, 17)


In [7]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a new top layer
x = base_model.output
predictions = Dense(17, activation='sigmoid')(x)

In [8]:
# let's visualize layer names and layer indices to see how many layers 
# we should freeze
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed0
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 average_pooling2d_2
51 conv2d_13
52 conv2d_15
53 conv2d_18
54 conv2d_19
55 batch_normalization_13
56 batch_norma

In [10]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in model.layers[:172]:
    layer.trainable = False
for layer in model.layers[172:]:
    layer.trainable = True

In [11]:
# Read in pre-trained weights - fast
# model.load_weights(obj_save_path + "weights_incv3.best.hdf5")

In [12]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer=SGD(lr=0.0001, momentum=0.9))

In [13]:
# Incorporate Callback features
# Checkpointing 
filepath= obj_save_path + "weights_incv3.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# Early Stopping
earlystop = EarlyStopping(monitor='val_loss', min_delta=0.0002, patience=5, verbose=0, mode='auto') 

callbacks_list = [checkpoint, earlystop]

In [14]:
# the explicit split approach was taken so as to allow for a local validation
# model.fit(X, Y, validation_split=0.33, epochs=150, batch_size=10, callbacks=callbacks_list, verbose=0)

# Fit the model (Add history so that the history may be saved)
history = model.fit(x_train, y_train,
          batch_size=128,
          epochs=1,
          verbose=1,
          callbacks=callbacks_list,
          validation_data=(x_valid, y_valid))

ValueError: Error when checking target: expected dense_1 to have 4 dimensions, but got array with shape (1024, 17)

In [ ]:
# Local Validation Results
from sklearn.metrics import fbeta_score
p_valid = model.predict(x_valid, batch_size=128,verbose=2)
print(fbeta_score(y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))

In [ ]:
# Plot Training History

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# purge the x_train and y_train dataset, and read in the test images
x_train = []
y_train = []

# x_test, y_test = keras_image_prep(test, test_path, 'test', input_dim, labels)
# Save out the test image NumPy
# np.save('f:/kaggle/amazon/x_test.npy', x_test)
# np.save('f:/kaggle/amazon/y_test.npy', y_test)

x_test = np.load('f:/kaggle/amazon/x_test.npy')
y_test = np.load('f:/kaggle/amazon/y_test.npy')

# predict using the trained model
y_test = model.predict(x_test,batch_size=128,verbose=2)
y_c_test = (y_test > .222222)*1.0        # before multiplying by 1.0, this appears as an array of True and False
print(y_c_test)

In [ ]:
# Build Submission, using label outputted from long time ago
test_labels = []
for row in range(y_c_test.shape[0]):
    test_labels.append(' '.join(labels[y_c_test[row,:]==1]))
Submission_PDFModel = test.copy()
Submission_PDFModel.drop('tags', axis = 1)
Submission_PDFModel['tags'] = test_labels
Submission_PDFModel.to_csv(str(subm_output_path + 'sub_simple_keras_32x32_rd6.csv'), index = False)

In [ ]:
# Trying out sampling of DataFrame
test_short = test.sample(frac=0.25,replace=False)
print(test_short.shape)

In [ ]:
print(labels)
print(Submission_PDFModel.shape)

In [ ]:
test_df = pd.DataFrame(history.history)                                # this is converting the dict object to DataFrame
test_df.to_csv(str(subm_output_path + 'keras_32x32_rd6_hist.csv'))     # saving to disk (a method for DataFrame object)
perserved_hist = pd.read_csv(str(subm_output_path + 'keras_32x32_rd6_hist.csv'))  # read in the preserved history

In [ ]:
# Plot (preserved) Training History

# summarize history for accuracy
plt.plot(perserved_hist['acc'])
plt.plot(perserved_hist['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(perserved_hist['loss'])
plt.plot(perserved_hist['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()